In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MaxAbsScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import get_scorer, plot_precision_recall_curve, precision_score, plot_roc_curve

sns.set_theme('poster')
pd.set_option('display.max_columns', None)

dir_fig = 'figures'

In [ ]:
import sklearn
sklearn.__version__

# Data Query Functions

## ASD

In [ ]:
def get_asd_demo():
    import pandas
    import os
    
    # This query represents dataset "ASD Rohini Spec" for domain "person" and was generated for All of Us Registered Tier Dataset v5
    dataset_76818386_person_sql = """
        SELECT
            person.SEX_AT_BIRTH_CONCEPT_ID,
            person.GENDER_CONCEPT_ID,
            person.BIRTH_DATETIME as DATE_OF_BIRTH,
            person.ETHNICITY_CONCEPT_ID,
            person.PERSON_ID,
            person.RACE_CONCEPT_ID,
            p_race_concept.concept_name as RACE,
            p_gender_concept.concept_name as GENDER,
            p_ethnicity_concept.concept_name as ETHNICITY,
            p_sex_at_birth_concept.concept_name as SEX_AT_BIRTH 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                on person.race_concept_id = p_race_concept.CONCEPT_ID 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                on person.gender_concept_id = p_gender_concept.CONCEPT_ID 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                on person.ethnicity_concept_id = p_ethnicity_concept.CONCEPT_ID 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                on person.sex_at_birth_concept_id = p_sex_at_birth_concept.CONCEPT_ID  
        WHERE
            person.PERSON_ID IN (
                SELECT
                    person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    is_standard = 1 
                                    AND concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    domain_id = 'CONDITION' 
                                                    AND is_standard = 1 
                                                    AND concept_id IN (439780, 439776) 
                                                    AND is_selectable = 1 
                                                    AND full_text LIKE '%[condition_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                domain_id = 'CONDITION' 
                                                AND is_standard = 1 
                                                AND is_selectable = 1
                                            )
                                    )
                                ) criteria 
                            UNION
                            ALL SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384458)
                                    )) criteria 
                            UNION
                            ALL SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        is_standard = 0 
                                        AND concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        domain_id = 'SURVEY' 
                                                        AND is_standard = 0 
                                                        AND concept_id IN (1384574, 1384630, 1384486) 
                                                        AND is_selectable = 1 
                                                        AND full_text LIKE '%[survey_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    domain_id = 'SURVEY' 
                                                    AND is_standard = 0 
                                                    AND is_selectable = 1
                                                )
                                        )
                                    ) criteria 
                                UNION
                                ALL SELECT
                                    criteria.person_id 
                                FROM
                                    (SELECT
                                        DISTINCT person_id,
                                        entry_date,
                                        concept_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                    WHERE
                                        (
                                            is_standard = 1 
                                            AND concept_id IN (
                                                SELECT
                                                    DISTINCT c.concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                JOIN
                                                    (
                                                        select
                                                            cast(cr.id as string) as id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                        WHERE
                                                            domain_id = 'CONDITION' 
                                                            AND is_standard = 1 
                                                            AND concept_id IN (45765723, 4254211, 439780, 439776, 439702, 439703) 
                                                            AND is_selectable = 1 
                                                            AND full_text LIKE '%[condition_rank1]%'
                                                    ) a 
                                                        ON (
                                                            c.path LIKE CONCAT('%.',
                                                        a.id,
                                                        '.%') 
                                                        OR c.path LIKE CONCAT('%.',
                                                        a.id) 
                                                        OR c.path LIKE CONCAT(a.id,
                                                        '.%') 
                                                        OR c.path = a.id) 
                                                    WHERE
                                                        domain_id = 'CONDITION' 
                                                        AND is_standard = 1 
                                                        AND is_selectable = 1
                                                    )
                                            )
                                        ) criteria 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            age_at_consent BETWEEN 65 AND 120 
                                    ) 
                                )"""

    dataset_76818386_person_df = pandas.read_gbq(
        dataset_76818386_person_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_76818386_person_df

In [ ]:
def get_asd_drug():
    import pandas
    import os
    
    # This query represents dataset "ASD Rohini Spec" for domain "drug" and was generated for All of Us Registered Tier Dataset v5
    dataset_76818386_drug_sql = """
        SELECT
            d_exposure.PERSON_ID,
            d_exposure.DRUG_SOURCE_CONCEPT_ID,
            d_exposure.QUANTITY,
            d_exposure.STOP_REASON,
            d_exposure.REFILLS,
            d_exposure.DRUG_TYPE_CONCEPT_ID,
            d_exposure.VISIT_OCCURRENCE_ID,
            d_exposure.ROUTE_CONCEPT_ID,
            d_exposure.DRUG_SOURCE_VALUE,
            d_exposure.SIG,
            d_exposure.ROUTE_SOURCE_VALUE,
            d_exposure.DRUG_CONCEPT_ID,
            d_exposure.DRUG_EXPOSURE_START_DATETIME,
            d_exposure.DAYS_SUPPLY,
            d_exposure.DRUG_EXPOSURE_END_DATETIME,
            d_exposure.VERBATIM_END_DATE,
            d_exposure.DOSE_UNIT_SOURCE_VALUE,
            d_exposure.LOT_NUMBER,
            d_route.concept_name as ROUTE_CONCEPT_NAME,
            d_type.concept_name as DRUG_TYPE_CONCEPT_NAME,
            d_standard_concept.vocabulary_id as STANDARD_VOCABULARY,
            d_standard_concept.concept_code as STANDARD_CONCEPT_CODE,
            d_standard_concept.concept_name as STANDARD_CONCEPT_NAME,
            d_source_concept.concept_name as SOURCE_CONCEPT_NAME,
            d_source_concept.concept_code as SOURCE_CONCEPT_CODE,
            d_source_concept.vocabulary_id as SOURCE_VOCABULARY,
            d_visit.concept_name as VISIT_OCCURRENCE_CONCEPT_NAME 
        FROM
            ( SELECT
                * 
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
            WHERE
                (
                    drug_concept_id IN  (
                        SELECT
                            DISTINCT ca.descendant_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                        JOIN
                            (
                                select
                                    distinct c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        select
                                            cast(cr.id as string) as id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                        WHERE
                                            domain_id = 'DRUG' 
                                            AND is_standard = 1 
                                            AND concept_id IN (
                                                21604181, 21604389, 21604801, 21604253, 21604685, 21604489, 21604443, 21603126, 21603089, 21601245, 21601278, 21603071, 21602796
                                            ) 
                                            AND is_selectable = 1 
                                            AND full_text LIKE '%[drug_rank1]%'
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        domain_id = 'DRUG' 
                                        AND is_standard = 1 
                                        AND is_selectable = 1
                                    ) b 
                                        ON (
                                            ca.ancestor_id = b.concept_id
                                        )
                                )
                            )  
                            AND (
                                d_exposure.PERSON_ID IN (
                                    SELECT
                                        person_id  
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 1 
                                                    AND concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND concept_id IN (439780, 439776) 
                                                                    AND is_selectable = 1 
                                                                    AND full_text LIKE '%[condition_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND is_selectable = 1
                                                            )
                                                    )
                                                ) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384458)
                                                    )) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 0 
                                                        AND concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        domain_id = 'SURVEY' 
                                                                        AND is_standard = 0 
                                                                        AND concept_id IN (1384574, 1384630, 1384486) 
                                                                        AND is_selectable = 1 
                                                                        AND full_text LIKE '%[survey_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    domain_id = 'SURVEY' 
                                                                    AND is_standard = 0 
                                                                    AND is_selectable = 1
                                                                )
                                                        )
                                                    ) criteria 
                                                UNION
                                                ALL SELECT
                                                    criteria.person_id 
                                                FROM
                                                    (SELECT
                                                        DISTINCT person_id,
                                                        entry_date,
                                                        concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                    WHERE
                                                        (
                                                            is_standard = 1 
                                                            AND concept_id IN (
                                                                SELECT
                                                                    DISTINCT c.concept_id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                                JOIN
                                                                    (
                                                                        select
                                                                            cast(cr.id as string) as id 
                                                                        FROM
                                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                        WHERE
                                                                            domain_id = 'CONDITION' 
                                                                            AND is_standard = 1 
                                                                            AND concept_id IN (45765723, 4254211, 439780, 439776, 439702, 439703) 
                                                                            AND is_selectable = 1 
                                                                            AND full_text LIKE '%[condition_rank1]%'
                                                                    ) a 
                                                                        ON (
                                                                            c.path LIKE CONCAT('%.',
                                                                        a.id,
                                                                        '.%') 
                                                                        OR c.path LIKE CONCAT('%.',
                                                                        a.id) 
                                                                        OR c.path LIKE CONCAT(a.id,
                                                                        '.%') 
                                                                        OR c.path = a.id) 
                                                                    WHERE
                                                                        domain_id = 'CONDITION' 
                                                                        AND is_standard = 1 
                                                                        AND is_selectable = 1
                                                                    )
                                                            )
                                                        ) criteria 
                                                    ) 
                                                    AND cb_search_person.person_id NOT IN (
                                                        SELECT
                                                            person_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                                        WHERE
                                                            age_at_consent BETWEEN 65 AND 120 
                                                    ) 
                                                ))) d_exposure 
                                        LEFT JOIN
                                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_route 
                                                on d_exposure.ROUTE_CONCEPT_ID = d_route.CONCEPT_ID 
                                        LEFT JOIN
                                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_type 
                                                on d_exposure.drug_type_concept_id = d_type.CONCEPT_ID 
                                        left join
                                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
                                                on d_exposure.DRUG_CONCEPT_ID = d_standard_concept.CONCEPT_ID 
                                        LEFT JOIN
                                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_source_concept 
                                                on d_exposure.DRUG_SOURCE_CONCEPT_ID = d_source_concept.CONCEPT_ID 
                                        left join
                                            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                                on d_exposure.VISIT_OCCURRENCE_ID = v.VISIT_OCCURRENCE_ID 
                                        LEFT JOIN
                                            `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_visit 
                                                on v.VISIT_CONCEPT_ID = d_visit.CONCEPT_ID"""

    dataset_76818386_drug_df = pandas.read_gbq(
        dataset_76818386_drug_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_76818386_drug_df

In [ ]:
def get_asd_labs():
    import pandas
    import os
    
    # This query represents dataset "ASD Rohini Spec" for domain "measurement" and was generated for All of Us Registered Tier Dataset v5
    dataset_76818386_measurement_sql = """
        SELECT
            measurement.RANGE_HIGH,
            measurement.MEASUREMENT_CONCEPT_ID,
            measurement.MEASUREMENT_DATETIME,
            measurement.MEASUREMENT_SOURCE_CONCEPT_ID,
            measurement.OPERATOR_CONCEPT_ID,
            measurement.VALUE_SOURCE_VALUE,
            measurement.MEASUREMENT_TYPE_CONCEPT_ID,
            measurement.VALUE_AS_CONCEPT_ID,
            measurement.PERSON_ID,
            measurement.MEASUREMENT_SOURCE_VALUE,
            measurement.RANGE_LOW,
            measurement.VALUE_AS_NUMBER,
            measurement.VISIT_OCCURRENCE_ID,
            measurement.UNIT_SOURCE_VALUE,
            measurement.UNIT_CONCEPT_ID,
            m_unit.concept_name as UNIT_CONCEPT_NAME,
            m_value.concept_name as VALUE_AS_CONCEPT_NAME,
            m_operator.concept_name as OPERATOR_CONCEPT_NAME,
            m_type.concept_name as MEASUREMENT_TYPE_CONCEPT_NAME,
            m_standard_concept.concept_name as STANDARD_CONCEPT_NAME,
            m_standard_concept.concept_code as STANDARD_CONCEPT_CODE,
            m_standard_concept.vocabulary_id as STANDARD_VOCABULARY,
            m_source_concept.concept_name as SOURCE_CONCEPT_NAME,
            m_source_concept.vocabulary_id as SOURCE_VOCABULARY,
            m_source_concept.concept_code as SOURCE_CONCEPT_CODE,
            m_visit.concept_name as VISIT_OCCURRENCE_CONCEPT_NAME 
        FROM
            ( SELECT
                * 
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
            WHERE
                (
                    measurement_concept_id IN  (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                select
                                    cast(cr.id as string) as id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                WHERE
                                    domain_id = 'MEASUREMENT' 
                                    AND is_standard = 1 
                                    AND concept_id IN (
                                        3027018, 3031203, 40765148, 3022318, 3036277, 3012888, 40759207, 3004249, 3025315, 3038553, 1011762, 40782741, 40779254, 40785816, 40776498, 37067286, 40783188, 37045941, 37066779, 40789305, 40789215, 1014209, 37066788, 40792336, 37069265, 40792772, 40785948, 40782666, 40779561, 40786228, 40779413, 40792787, 40779574, 40782929, 40786231, 40796104, 40779579, 40783033, 40779580, 40796100, 40772938, 40797982, 40772940, 40772939, 40772941, 40789383, 40792777, 40782926, 40792618, 40772936, 40786222, 40792784, 40795954, 1002664, 37023425, 37032269, 2213001, 2212980, 3023368, 3007682, 3013184, 37073102, 3005058, 3017753, 3000144, 3000074, 3019894, 3013362, 37035794, 42868623, 3028707, 42868621, 3018060, 3044376, 37061616, 3018756, 3015743, 42868627, 3021016, 42868629, 43533393, 3012592, 3006932, 3023596, 43533388, 42868624, 3015208
                                    ) 
                                    AND is_selectable = 1 
                                    AND full_text LIKE '%[measurement_rank1]%'
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                domain_id = 'MEASUREMENT' 
                                AND is_standard = 1 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        measurement.PERSON_ID IN (
                            SELECT
                                person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                is_standard = 1 
                                                AND concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND concept_id IN (439780, 439776) 
                                                                AND is_selectable = 1 
                                                                AND full_text LIKE '%[condition_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            domain_id = 'CONDITION' 
                                                            AND is_standard = 1 
                                                            AND is_selectable = 1
                                                        )
                                                )
                                            ) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384458)
                                                )) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    domain_id = 'SURVEY' 
                                                                    AND is_standard = 0 
                                                                    AND concept_id IN (1384574, 1384630, 1384486) 
                                                                    AND is_selectable = 1 
                                                                    AND full_text LIKE '%[survey_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                domain_id = 'SURVEY' 
                                                                AND is_standard = 0 
                                                                AND is_selectable = 1
                                                            )
                                                    )
                                                ) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 1 
                                                        AND concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        domain_id = 'CONDITION' 
                                                                        AND is_standard = 1 
                                                                        AND concept_id IN (45765723, 4254211, 439780, 439776, 439702, 439703) 
                                                                        AND is_selectable = 1 
                                                                        AND full_text LIKE '%[condition_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND is_selectable = 1
                                                                )
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        person_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                                    WHERE
                                                        age_at_consent BETWEEN 65 AND 120 
                                                ) 
                                            ))) measurement 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
                                            on measurement.unit_concept_id = m_unit.concept_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_value 
                                            on measurement.value_as_concept_id = m_value.concept_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_operator 
                                            on measurement.operator_concept_id = m_operator.concept_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_type 
                                            on measurement.measurement_type_concept_id = m_type.concept_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
                                            on measurement.measurement_concept_id = m_standard_concept.concept_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_source_concept 
                                            on measurement.measurement_source_concept_id = m_source_concept.concept_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                            on measurement.visit_occurrence_id = v.visit_occurrence_id 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_visit 
                                            on v.visit_concept_id = m_visit.concept_id"""

    dataset_76818386_measurement_df = pandas.read_gbq(
        dataset_76818386_measurement_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_76818386_measurement_df

In [ ]:
def get_asd_proc():
    import pandas
    import os

    # This query represents dataset "ASD Rohini Spec" for domain "procedure" and was generated for All of Us Registered Tier Dataset v5
    dataset_76818386_procedure_sql = """
        SELECT
            procedure.PROCEDURE_SOURCE_VALUE,
            procedure.PROCEDURE_CONCEPT_ID,
            procedure.PROCEDURE_DATETIME,
            procedure.QUANTITY,
            procedure.PROCEDURE_TYPE_CONCEPT_ID,
            procedure.VISIT_OCCURRENCE_ID,
            procedure.MODIFIER_CONCEPT_ID,
            procedure.QUALIFIER_SOURCE_VALUE,
            procedure.PERSON_ID,
            procedure.PROCEDURE_SOURCE_CONCEPT_ID,
            p_type.concept_name as PROCEDURE_TYPE_CONCEPT_NAME,
            p_modifier.concept_name as MODIFIER_CONCEPT_NAME,
            p_standard_concept.concept_code as STANDARD_CONCEPT_CODE,
            p_standard_concept.concept_name as STANDARD_CONCEPT_NAME,
            p_standard_concept.vocabulary_id as STANDARD_VOCABULARY,
            p_source_concept.concept_code as SOURCE_CONCEPT_CODE,
            p_source_concept.vocabulary_id as SOURCE_VOCABULARY,
            p_source_concept.concept_name as SOURCE_CONCEPT_NAME,
            p_visit.concept_name as VISIT_OCCURRENCE_CONCEPT_NAME 
        FROM
            ( SELECT
                * 
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.procedure_occurrence` procedure 
            WHERE
                (
                    procedure_concept_id IN  (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                select
                                    cast(cr.id as string) as id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                WHERE
                                    domain_id = 'PROCEDURE' 
                                    AND is_standard = 1 
                                    AND concept_id IN (
                                        4080549, 4163872, 4125350, 40481383, 4058336, 4061009
                                    ) 
                                    AND is_selectable = 1 
                                    AND full_text LIKE '%[procedure_rank1]%'
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                domain_id = 'PROCEDURE' 
                                AND is_standard = 1 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        procedure.PERSON_ID IN (
                            SELECT
                                person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                is_standard = 1 
                                                AND concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND concept_id IN (439780, 439776) 
                                                                AND is_selectable = 1 
                                                                AND full_text LIKE '%[condition_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            domain_id = 'CONDITION' 
                                                            AND is_standard = 1 
                                                            AND is_selectable = 1
                                                        )
                                                )
                                            ) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384458)
                                                )) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    domain_id = 'SURVEY' 
                                                                    AND is_standard = 0 
                                                                    AND concept_id IN (1384574, 1384630, 1384486) 
                                                                    AND is_selectable = 1 
                                                                    AND full_text LIKE '%[survey_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                domain_id = 'SURVEY' 
                                                                AND is_standard = 0 
                                                                AND is_selectable = 1
                                                            )
                                                    )
                                                ) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 1 
                                                        AND concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        domain_id = 'CONDITION' 
                                                                        AND is_standard = 1 
                                                                        AND concept_id IN (45765723, 4254211, 439780, 439776, 439702, 439703) 
                                                                        AND is_selectable = 1 
                                                                        AND full_text LIKE '%[condition_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND is_selectable = 1
                                                                )
                                                        )
                                                    ) criteria 
                                                ) 
                                                AND cb_search_person.person_id NOT IN (
                                                    SELECT
                                                        person_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                                    WHERE
                                                        age_at_consent BETWEEN 65 AND 120 
                                                ) 
                                            ))) procedure 
                                    LEFT JOIN
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_type 
                                            on procedure.PROCEDURE_TYPE_CONCEPT_ID = p_type.CONCEPT_ID 
                                    LEFT JOIN
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_modifier 
                                            on procedure.MODIFIER_CONCEPT_ID = p_modifier.CONCEPT_ID 
                                    LEFT JOIN
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_standard_concept 
                                            on procedure.PROCEDURE_CONCEPT_ID = p_standard_concept.CONCEPT_ID 
                                    LEFT JOIN
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_source_concept 
                                            on procedure.PROCEDURE_SOURCE_CONCEPT_ID = p_source_concept.CONCEPT_ID 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                            on procedure.VISIT_OCCURRENCE_ID = v.VISIT_OCCURRENCE_ID 
                                    left join
                                        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_visit 
                                            on v.visit_concept_id = p_visit.concept_id"""

    dataset_76818386_procedure_df = pandas.read_gbq(
        dataset_76818386_procedure_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_76818386_procedure_df

## non-ASD

In [ ]:
def get_ctrl_demo():
    import pandas
    import os

    # This query represents dataset "non-ASD Rohini Spec" for domain "person" and was generated for All of Us Registered Tier Dataset v5
    dataset_30052106_person_sql = """
        SELECT
            person.SEX_AT_BIRTH_CONCEPT_ID,
            person.GENDER_CONCEPT_ID,
            person.BIRTH_DATETIME as DATE_OF_BIRTH,
            person.ETHNICITY_CONCEPT_ID,
            person.PERSON_ID,
            person.RACE_CONCEPT_ID,
            p_race_concept.concept_name as RACE,
            p_gender_concept.concept_name as GENDER,
            p_ethnicity_concept.concept_name as ETHNICITY,
            p_sex_at_birth_concept.concept_name as SEX_AT_BIRTH 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
                on person.race_concept_id = p_race_concept.CONCEPT_ID 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
                on person.gender_concept_id = p_gender_concept.CONCEPT_ID 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
                on person.ethnicity_concept_id = p_ethnicity_concept.CONCEPT_ID 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
                on person.sex_at_birth_concept_id = p_sex_at_birth_concept.CONCEPT_ID  
        WHERE
            person.PERSON_ID IN (
                SELECT
                    person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            person_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                        WHERE
                            age_at_consent BETWEEN 18 AND 65 
                    ) 
                    AND cb_search_person.person_id NOT IN (
                        SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    is_standard = 1 
                                    AND concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                WHERE
                                                    domain_id = 'CONDITION' 
                                                    AND is_standard = 1 
                                                    AND concept_id IN (45765723, 4254211, 439780, 439776, 4053178, 439702, 43020503, 439703) 
                                                    AND is_selectable = 1 
                                                    AND full_text LIKE '%[condition_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                domain_id = 'CONDITION' 
                                                AND is_standard = 1 
                                                AND is_selectable = 1
                                            )
                                    )
                                ) criteria 
                            UNION
                            ALL SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384458) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384630)  
                                        AND  value_source_concept_id IN (1384998) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384574)  
                                        AND  value_source_concept_id IN (1385247) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384574)  
                                        AND  value_source_concept_id IN (1385384) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384574)  
                                        AND  value_source_concept_id IN (1384787) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384574)  
                                        AND  value_source_concept_id IN (1384815) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384574)  
                                        AND  value_source_concept_id IN (1385326) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384486)  
                                        AND  value_source_concept_id IN (1385369)
                                    )) criteria 
                            UNION
                            ALL SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384600) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384475) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384510) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384458) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384669) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384656) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384413) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384558) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384622) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384443) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384557) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384464) 
                                        OR  is_standard = 0 
                                        AND concept_id IN (1384495)  
                                        AND  value_source_concept_id IN (1384549)
                                    )) criteria 
                            UNION
                            ALL SELECT
                                criteria.person_id 
                            FROM
                                (SELECT
                                    DISTINCT person_id,
                                    entry_date,
                                    concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                WHERE
                                    (
                                        is_standard = 1 
                                        AND concept_id IN (
                                            SELECT
                                                DISTINCT c.concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                            JOIN
                                                (
                                                    select
                                                        cast(cr.id as string) as id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                    WHERE
                                                        domain_id = 'CONDITION' 
                                                        AND is_standard = 1 
                                                        AND concept_id IN (45771096, 432586, 43530900, 4279309) 
                                                        AND is_selectable = 1 
                                                        AND full_text LIKE '%[condition_rank1]%'
                                                ) a 
                                                    ON (
                                                        c.path LIKE CONCAT('%.',
                                                    a.id,
                                                    '.%') 
                                                    OR c.path LIKE CONCAT('%.',
                                                    a.id) 
                                                    OR c.path LIKE CONCAT(a.id,
                                                    '.%') 
                                                    OR c.path = a.id) 
                                                WHERE
                                                    domain_id = 'CONDITION' 
                                                    AND is_standard = 1 
                                                    AND is_selectable = 1
                                                )
                                        )
                                    ) criteria 
                                ) )"""

    dataset_30052106_person_df = pandas.read_gbq(
        dataset_30052106_person_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_30052106_person_df

In [ ]:
def get_ctrl_drug():
    import pandas
    import os

    # This query represents dataset "non-ASD Rohini Spec" for domain "drug" and was generated for All of Us Registered Tier Dataset v5
    dataset_30052106_drug_sql = """
        SELECT
            d_exposure.PERSON_ID,
            d_exposure.DRUG_SOURCE_CONCEPT_ID,
            d_exposure.QUANTITY,
            d_exposure.STOP_REASON,
            d_exposure.REFILLS,
            d_exposure.DRUG_TYPE_CONCEPT_ID,
            d_exposure.VISIT_OCCURRENCE_ID,
            d_exposure.ROUTE_CONCEPT_ID,
            d_exposure.DRUG_SOURCE_VALUE,
            d_exposure.SIG,
            d_exposure.ROUTE_SOURCE_VALUE,
            d_exposure.DRUG_CONCEPT_ID,
            d_exposure.DRUG_EXPOSURE_START_DATETIME,
            d_exposure.DAYS_SUPPLY,
            d_exposure.DRUG_EXPOSURE_END_DATETIME,
            d_exposure.VERBATIM_END_DATE,
            d_exposure.DOSE_UNIT_SOURCE_VALUE,
            d_exposure.LOT_NUMBER,
            d_route.concept_name as ROUTE_CONCEPT_NAME,
            d_type.concept_name as DRUG_TYPE_CONCEPT_NAME,
            d_standard_concept.vocabulary_id as STANDARD_VOCABULARY,
            d_standard_concept.concept_code as STANDARD_CONCEPT_CODE,
            d_standard_concept.concept_name as STANDARD_CONCEPT_NAME,
            d_source_concept.concept_name as SOURCE_CONCEPT_NAME,
            d_source_concept.concept_code as SOURCE_CONCEPT_CODE,
            d_source_concept.vocabulary_id as SOURCE_VOCABULARY,
            d_visit.concept_name as VISIT_OCCURRENCE_CONCEPT_NAME 
        FROM
            ( SELECT
                * 
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
            WHERE
                (
                    drug_concept_id IN  (
                        SELECT
                            DISTINCT ca.descendant_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                        JOIN
                            (
                                select
                                    distinct c.concept_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                JOIN
                                    (
                                        select
                                            cast(cr.id as string) as id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                        WHERE
                                            domain_id = 'DRUG' 
                                            AND is_standard = 1 
                                            AND concept_id IN (
                                                21604181, 21604389, 21604801, 21604253, 21604685, 21604489, 21604443, 21603126, 21603089, 21601245, 21601278, 21603071, 21602796
                                            ) 
                                            AND is_selectable = 1 
                                            AND full_text LIKE '%[drug_rank1]%'
                                    ) a 
                                        ON (
                                            c.path LIKE CONCAT('%.',
                                        a.id,
                                        '.%') 
                                        OR c.path LIKE CONCAT('%.',
                                        a.id) 
                                        OR c.path LIKE CONCAT(a.id,
                                        '.%') 
                                        OR c.path = a.id) 
                                    WHERE
                                        domain_id = 'DRUG' 
                                        AND is_standard = 1 
                                        AND is_selectable = 1
                                    ) b 
                                        ON (
                                            ca.ancestor_id = b.concept_id
                                        )
                                )
                            )  
                            AND (
                                d_exposure.PERSON_ID IN (
                                    SELECT
                                        person_id  
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                                WHERE
                                    cb_search_person.person_id IN (
                                        SELECT
                                            person_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                        WHERE
                                            age_at_consent BETWEEN 18 AND 65 
                                    ) 
                                    AND cb_search_person.person_id NOT IN (
                                        SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 1 
                                                    AND concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND concept_id IN (45765723, 4254211, 439780, 439776, 4053178, 439702, 43020503, 439703) 
                                                                    AND is_selectable = 1 
                                                                    AND full_text LIKE '%[condition_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND is_selectable = 1
                                                            )
                                                    )
                                                ) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384458) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384630)  
                                                        AND  value_source_concept_id IN (1384998) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384574)  
                                                        AND  value_source_concept_id IN (1385247) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384574)  
                                                        AND  value_source_concept_id IN (1385384) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384574)  
                                                        AND  value_source_concept_id IN (1384787) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384574)  
                                                        AND  value_source_concept_id IN (1384815) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384574)  
                                                        AND  value_source_concept_id IN (1385326) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384486)  
                                                        AND  value_source_concept_id IN (1385369)
                                                    )) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384600) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384475) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384510) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384458) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384669) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384656) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384413) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384558) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384622) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384443) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384557) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384464) 
                                                        OR  is_standard = 0 
                                                        AND concept_id IN (1384495)  
                                                        AND  value_source_concept_id IN (1384549)
                                                    )) criteria 
                                            UNION
                                            ALL SELECT
                                                criteria.person_id 
                                            FROM
                                                (SELECT
                                                    DISTINCT person_id,
                                                    entry_date,
                                                    concept_id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                                WHERE
                                                    (
                                                        is_standard = 1 
                                                        AND concept_id IN (
                                                            SELECT
                                                                DISTINCT c.concept_id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                            JOIN
                                                                (
                                                                    select
                                                                        cast(cr.id as string) as id 
                                                                    FROM
                                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                    WHERE
                                                                        domain_id = 'CONDITION' 
                                                                        AND is_standard = 1 
                                                                        AND concept_id IN (45771096, 432586, 43530900, 4279309) 
                                                                        AND is_selectable = 1 
                                                                        AND full_text LIKE '%[condition_rank1]%'
                                                                ) a 
                                                                    ON (
                                                                        c.path LIKE CONCAT('%.',
                                                                    a.id,
                                                                    '.%') 
                                                                    OR c.path LIKE CONCAT('%.',
                                                                    a.id) 
                                                                    OR c.path LIKE CONCAT(a.id,
                                                                    '.%') 
                                                                    OR c.path = a.id) 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND is_selectable = 1
                                                                )
                                                        )
                                                    ) criteria 
                                                ) ))
                                    ) d_exposure 
                                LEFT JOIN
                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_route 
                                        on d_exposure.ROUTE_CONCEPT_ID = d_route.CONCEPT_ID 
                                LEFT JOIN
                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_type 
                                        on d_exposure.drug_type_concept_id = d_type.CONCEPT_ID 
                                left join
                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
                                        on d_exposure.DRUG_CONCEPT_ID = d_standard_concept.CONCEPT_ID 
                                LEFT JOIN
                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_source_concept 
                                        on d_exposure.DRUG_SOURCE_CONCEPT_ID = d_source_concept.CONCEPT_ID 
                                left join
                                    `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                        on d_exposure.VISIT_OCCURRENCE_ID = v.VISIT_OCCURRENCE_ID 
                                LEFT JOIN
                                    `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_visit 
                                        on v.VISIT_CONCEPT_ID = d_visit.CONCEPT_ID"""

    dataset_30052106_drug_df = pandas.read_gbq(
        dataset_30052106_drug_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_30052106_drug_df

In [ ]:
def get_ctrl_labs():
    import pandas
    import os

    # This query represents dataset "non-ASD Rohini Spec" for domain "measurement" and was generated for All of Us Registered Tier Dataset v5
    dataset_30052106_measurement_sql = """
        SELECT
            measurement.RANGE_HIGH,
            measurement.MEASUREMENT_CONCEPT_ID,
            measurement.MEASUREMENT_DATETIME,
            measurement.MEASUREMENT_SOURCE_CONCEPT_ID,
            measurement.OPERATOR_CONCEPT_ID,
            measurement.VALUE_SOURCE_VALUE,
            measurement.MEASUREMENT_TYPE_CONCEPT_ID,
            measurement.VALUE_AS_CONCEPT_ID,
            measurement.PERSON_ID,
            measurement.MEASUREMENT_SOURCE_VALUE,
            measurement.RANGE_LOW,
            measurement.VALUE_AS_NUMBER,
            measurement.VISIT_OCCURRENCE_ID,
            measurement.UNIT_SOURCE_VALUE,
            measurement.UNIT_CONCEPT_ID,
            m_unit.concept_name as UNIT_CONCEPT_NAME,
            m_value.concept_name as VALUE_AS_CONCEPT_NAME,
            m_operator.concept_name as OPERATOR_CONCEPT_NAME,
            m_type.concept_name as MEASUREMENT_TYPE_CONCEPT_NAME,
            m_standard_concept.concept_name as STANDARD_CONCEPT_NAME,
            m_standard_concept.concept_code as STANDARD_CONCEPT_CODE,
            m_standard_concept.vocabulary_id as STANDARD_VOCABULARY,
            m_source_concept.concept_name as SOURCE_CONCEPT_NAME,
            m_source_concept.vocabulary_id as SOURCE_VOCABULARY,
            m_source_concept.concept_code as SOURCE_CONCEPT_CODE,
            m_visit.concept_name as VISIT_OCCURRENCE_CONCEPT_NAME 
        FROM
            ( SELECT
                * 
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.measurement` measurement 
            WHERE
                (
                    measurement_concept_id IN  (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                select
                                    cast(cr.id as string) as id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                WHERE
                                    domain_id = 'MEASUREMENT' 
                                    AND is_standard = 1 
                                    AND concept_id IN (
                                        3027018, 3031203, 40765148, 3022318, 3036277, 3012888, 40759207, 3004249, 3025315, 3038553, 1011762, 40782741, 40779254, 40785816, 40776498, 37067286, 40783188, 37045941, 37066779, 40789305, 40789215, 1014209, 37066788, 40792336, 37069265, 40792772, 40785948, 40782666, 40779561, 40786228, 40779413, 40792787, 40779574, 40782929, 40786231, 40796104, 40779579, 40783033, 40779580, 40796100, 40772938, 40797982, 40772940, 40772939, 40772941, 40789383, 40792777, 40782926, 40792618, 40772936, 40786222, 40792784, 40795954, 1002664, 37023425, 37032269, 2213001, 2212980, 3023368, 3007682, 3013184, 37073102, 3005058, 3017753, 3000144, 3000074, 3019894, 3013362, 37035794, 42868623, 3028707, 42868621, 3018060, 3044376, 37061616, 3018756, 3015743, 42868627, 3021016, 42868629, 43533393, 3012592, 3006932, 3023596, 43533388, 42868624, 3015208
                                    ) 
                                    AND is_selectable = 1 
                                    AND full_text LIKE '%[measurement_rank1]%'
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                domain_id = 'MEASUREMENT' 
                                AND is_standard = 1 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        measurement.PERSON_ID IN (
                            SELECT
                                person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_consent BETWEEN 18 AND 65 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                is_standard = 1 
                                                AND concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND concept_id IN (45765723, 4254211, 439780, 439776, 4053178, 439702, 43020503, 439703) 
                                                                AND is_selectable = 1 
                                                                AND full_text LIKE '%[condition_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            domain_id = 'CONDITION' 
                                                            AND is_standard = 1 
                                                            AND is_selectable = 1
                                                        )
                                                )
                                            ) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384458) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384630)  
                                                    AND  value_source_concept_id IN (1384998) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1385247) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1385384) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1384787) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1384815) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1385326) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384486)  
                                                    AND  value_source_concept_id IN (1385369)
                                                )) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384600) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384475) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384510) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384458) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384669) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384656) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384413) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384558) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384622) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384443) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384557) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384464) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384549)
                                                )) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 1 
                                                    AND concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND concept_id IN (45771096, 432586, 43530900, 4279309) 
                                                                    AND is_selectable = 1 
                                                                    AND full_text LIKE '%[condition_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND is_selectable = 1
                                                            )
                                                    )
                                                ) criteria 
                                            ) ))
                                ) measurement 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_unit 
                                    on measurement.unit_concept_id = m_unit.concept_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_value 
                                    on measurement.value_as_concept_id = m_value.concept_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_operator 
                                    on measurement.operator_concept_id = m_operator.concept_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_type 
                                    on measurement.measurement_type_concept_id = m_type.concept_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_standard_concept 
                                    on measurement.measurement_concept_id = m_standard_concept.concept_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_source_concept 
                                    on measurement.measurement_source_concept_id = m_source_concept.concept_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                    on measurement.visit_occurrence_id = v.visit_occurrence_id 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` m_visit 
                                    on v.visit_concept_id = m_visit.concept_id"""

    dataset_30052106_measurement_df = pandas.read_gbq(
        dataset_30052106_measurement_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_30052106_measurement_df

In [ ]:
def get_ctrl_proc():
    import pandas
    import os

    # This query represents dataset "non-ASD Rohini Spec" for domain "procedure" and was generated for All of Us Registered Tier Dataset v5
    dataset_30052106_procedure_sql = """
        SELECT
            procedure.PROCEDURE_SOURCE_VALUE,
            procedure.PROCEDURE_CONCEPT_ID,
            procedure.PROCEDURE_DATETIME,
            procedure.QUANTITY,
            procedure.PROCEDURE_TYPE_CONCEPT_ID,
            procedure.VISIT_OCCURRENCE_ID,
            procedure.MODIFIER_CONCEPT_ID,
            procedure.QUALIFIER_SOURCE_VALUE,
            procedure.PERSON_ID,
            procedure.PROCEDURE_SOURCE_CONCEPT_ID,
            p_type.concept_name as PROCEDURE_TYPE_CONCEPT_NAME,
            p_modifier.concept_name as MODIFIER_CONCEPT_NAME,
            p_standard_concept.concept_code as STANDARD_CONCEPT_CODE,
            p_standard_concept.concept_name as STANDARD_CONCEPT_NAME,
            p_standard_concept.vocabulary_id as STANDARD_VOCABULARY,
            p_source_concept.concept_code as SOURCE_CONCEPT_CODE,
            p_source_concept.vocabulary_id as SOURCE_VOCABULARY,
            p_source_concept.concept_name as SOURCE_CONCEPT_NAME,
            p_visit.concept_name as VISIT_OCCURRENCE_CONCEPT_NAME 
        FROM
            ( SELECT
                * 
            from
                `""" + os.environ["WORKSPACE_CDR"] + """.procedure_occurrence` procedure 
            WHERE
                (
                    procedure_concept_id IN  (
                        SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (
                                select
                                    cast(cr.id as string) as id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                WHERE
                                    domain_id = 'PROCEDURE' 
                                    AND is_standard = 1 
                                    AND concept_id IN (
                                        4080549, 4163872, 4125350, 40481383, 4058336, 4061009
                                    ) 
                                    AND is_selectable = 1 
                                    AND full_text LIKE '%[procedure_rank1]%'
                            ) a 
                                ON (
                                    c.path LIKE CONCAT('%.',
                                a.id,
                                '.%') 
                                OR c.path LIKE CONCAT('%.',
                                a.id) 
                                OR c.path LIKE CONCAT(a.id,
                                '.%') 
                                OR c.path = a.id) 
                            WHERE
                                domain_id = 'PROCEDURE' 
                                AND is_standard = 1 
                                AND is_selectable = 1
                            )
                    )  
                    AND (
                        procedure.PERSON_ID IN (
                            SELECT
                                person_id  
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                            WHERE
                                cb_search_person.person_id IN (
                                    SELECT
                                        person_id 
                                    FROM
                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                    WHERE
                                        age_at_consent BETWEEN 18 AND 65 
                                ) 
                                AND cb_search_person.person_id NOT IN (
                                    SELECT
                                        criteria.person_id 
                                    FROM
                                        (SELECT
                                            DISTINCT person_id,
                                            entry_date,
                                            concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                        WHERE
                                            (
                                                is_standard = 1 
                                                AND concept_id IN (
                                                    SELECT
                                                        DISTINCT c.concept_id 
                                                    FROM
                                                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                    JOIN
                                                        (
                                                            select
                                                                cast(cr.id as string) as id 
                                                            FROM
                                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND concept_id IN (45765723, 4254211, 439780, 439776, 4053178, 439702, 43020503, 439703) 
                                                                AND is_selectable = 1 
                                                                AND full_text LIKE '%[condition_rank1]%'
                                                        ) a 
                                                            ON (
                                                                c.path LIKE CONCAT('%.',
                                                            a.id,
                                                            '.%') 
                                                            OR c.path LIKE CONCAT('%.',
                                                            a.id) 
                                                            OR c.path LIKE CONCAT(a.id,
                                                            '.%') 
                                                            OR c.path = a.id) 
                                                        WHERE
                                                            domain_id = 'CONDITION' 
                                                            AND is_standard = 1 
                                                            AND is_selectable = 1
                                                        )
                                                )
                                            ) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384458) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384630)  
                                                    AND  value_source_concept_id IN (1384998) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1385247) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1385384) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1384787) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1384815) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384574)  
                                                    AND  value_source_concept_id IN (1385326) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384486)  
                                                    AND  value_source_concept_id IN (1385369)
                                                )) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384600) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384475) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384510) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384458) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384669) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384656) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384413) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384558) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384622) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384443) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384557) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384464) 
                                                    OR  is_standard = 0 
                                                    AND concept_id IN (1384495)  
                                                    AND  value_source_concept_id IN (1384549)
                                                )) criteria 
                                        UNION
                                        ALL SELECT
                                            criteria.person_id 
                                        FROM
                                            (SELECT
                                                DISTINCT person_id,
                                                entry_date,
                                                concept_id 
                                            FROM
                                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                                            WHERE
                                                (
                                                    is_standard = 1 
                                                    AND concept_id IN (
                                                        SELECT
                                                            DISTINCT c.concept_id 
                                                        FROM
                                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                                        JOIN
                                                            (
                                                                select
                                                                    cast(cr.id as string) as id 
                                                                FROM
                                                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr 
                                                                WHERE
                                                                    domain_id = 'CONDITION' 
                                                                    AND is_standard = 1 
                                                                    AND concept_id IN (45771096, 432586, 43530900, 4279309) 
                                                                    AND is_selectable = 1 
                                                                    AND full_text LIKE '%[condition_rank1]%'
                                                            ) a 
                                                                ON (
                                                                    c.path LIKE CONCAT('%.',
                                                                a.id,
                                                                '.%') 
                                                                OR c.path LIKE CONCAT('%.',
                                                                a.id) 
                                                                OR c.path LIKE CONCAT(a.id,
                                                                '.%') 
                                                                OR c.path = a.id) 
                                                            WHERE
                                                                domain_id = 'CONDITION' 
                                                                AND is_standard = 1 
                                                                AND is_selectable = 1
                                                            )
                                                    )
                                                ) criteria 
                                            ) ))
                                ) procedure 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_type 
                                    on procedure.PROCEDURE_TYPE_CONCEPT_ID = p_type.CONCEPT_ID 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_modifier 
                                    on procedure.MODIFIER_CONCEPT_ID = p_modifier.CONCEPT_ID 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_standard_concept 
                                    on procedure.PROCEDURE_CONCEPT_ID = p_standard_concept.CONCEPT_ID 
                            LEFT JOIN
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_source_concept 
                                    on procedure.PROCEDURE_SOURCE_CONCEPT_ID = p_source_concept.CONCEPT_ID 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                                    on procedure.VISIT_OCCURRENCE_ID = v.VISIT_OCCURRENCE_ID 
                            left join
                                `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_visit 
                                    on v.visit_concept_id = p_visit.concept_id"""

    dataset_30052106_procedure_df = pandas.read_gbq(
        dataset_30052106_procedure_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")

    return dataset_30052106_procedure_df

# Functions

In [ ]:
def check_memory_usage():
    import os, psutil
    process = psutil.Process()
    print(f'Using {process.memory_info().rss/1024**2:.3f} Mb.')

In [ ]:
def query_aou(sql: str) -> pd.DataFrame:
    return pd.read_gbq(
        sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook"
    )

In [ ]:
def get_concepts():
    sql = f"""
    SELECT *
    FROM {os.environ['WORKSPACE_CDR']}.concept
    """

    return query_aou(sql)

In [ ]:
def get_ancestors(concept_ids):
    """Retrieve CONCEPT_ANCESTOR dataframe containing ancestors of provided concepts."""
    
    def _check_if_iterable(obj):
        try: _ = iter(obj)
        except TypeError: return False
        else: return True
    
    # Reduce to unique concepts and apply formatting for SQL
    if isinstance(concept_ids, pd.Index):
        concept_ids = concept_ids.astype(str)
    elif isinstance(concept_ids, pd.Series):
        concept_ids = concept_ids.unique().astype(str)
    elif isinstance(concept_ids, np.ndarray):
        concept_ids = np.unique(concept_ids.astype(str))
    elif not isinstance(concept_ids, str) and _check_if_iterable(concept_ids):
        concept_ids = (str(c) for c in concept_ids)
        
    if not isinstance(concept_ids, str):
        concept_ids = ', '.join(concept_ids)

    sql = f"""
    SELECT 
        ca.*,
        c_anc.concept_name AS ancestor_name,
        c_anc.standard_concept AS ancestor_class,
        c_desc.concept_name AS descendant_name,
        c_desc.standard_concept AS descendant_class
    FROM {os.environ["WORKSPACE_CDR"]}.concept_ancestor AS ca
    JOIN {os.environ["WORKSPACE_CDR"]}.concept AS c_anc
    ON c_anc.concept_id = ca.ancestor_concept_id
    JOIN {os.environ["WORKSPACE_CDR"]}.concept AS c_desc
    ON c_desc.concept_id = ca.descendant_concept_id
    WHERE (
        ca.descendant_concept_id IN ({concept_ids})
        AND min_levels_of_separation > 0
    )
    """
    
    df = query_aou(sql)
    
    return df

In [ ]:
def get_per_subject_freqs(df, freq_col):
    new_df = (
        df
        .groupby(['PERSON_ID', freq_col])
        .size()
        .to_frame(name='count')
        .reset_index()
        .pivot_table(index='PERSON_ID', columns=freq_col, values='count', fill_value=0)
    )
    return new_df

In [ ]:
def flip_binary_labels(labels):
    flipped = labels.copy()
    flipped -= 1
    flipped = abs(flipped)
    return flipped

In [ ]:
def neg_pred_val(y_true, y_pred):
    y_true_flip = flip_binary_labels(y_true)
    y_pred_flip = flip_binary_labels(y_pred)
    return precision_score(y_true_flip, y_pred_flip)

In [ ]:
def clf_metrics(clf, X_train, X_test, y_train, y_test):
    """Report PPV, NPV, AUPRC, and F1 score on train and test sets."""    
    scorer_skl = ['precision', 'average_precision', 'roc_auc', 'f1', 'f1_macro', 'balanced_accuracy']
    scorers = [get_scorer(skl) for skl in scorer_skl]
    scorer_names = ['PPV', 'AUPRC', 'AUROC', 'F1', 'F1 Macro', 'Balanced Accuracy']
    metric_data = {name: [scorer(clf, X_train, y_train), scorer(clf, X_test, y_test)] for scorer, name in zip(scorers, scorer_names)}
    metric_data['NPV'] = [neg_pred_val(y_train, clf.predict(X_train)), neg_pred_val(y_test, clf.predict(X_test))]
    
    df_metric = pd.DataFrame.from_dict(metric_data, orient='index', columns=['Train', 'Test'])
    
    return(df_metric)

# Analysis

## Labs & Measurements

### Loading & Cleaning

In [ ]:
asd_labs = get_asd_labs()
useful_cols = ['PERSON_ID', 'MEASUREMENT_CONCEPT_ID', 'MEASUREMENT_DATETIME', 'STANDARD_CONCEPT_NAME', 'VALUE_AS_NUMBER',
               'RANGE_LOW', 'RANGE_HIGH', 'UNIT_SOURCE_VALUE', 'UNIT_CONCEPT_NAME', 'MEASUREMENT_TYPE_CONCEPT_NAME']
asd_labs = asd_labs[useful_cols]

In [ ]:
asd_lab_anc = get_ancestors(asd_labs['MEASUREMENT_CONCEPT_ID'])

In [ ]:
asd_lab_to_class = asd_lab_anc.loc[(
    asd_lab_anc[asd_lab_anc['ancestor_class'] == 'C']
    .groupby('descendant_concept_id')['min_levels_of_separation']
    .idxmin()
)]

In [ ]:
asd_lab_to_class['ancestor_name'].value_counts()

### Summary

In [ ]:
asd_labs_subj_freqs = get_per_subject_freqs(asd_labs, 'MEASUREMENT_CONCEPT_ID')
asd_labs_freqs = asd_labs.groupby('STANDARD_CONCEPT_NAME').size().sort_values(ascending=False)

In [ ]:
# Summarize frequencies of top k measurements
k = 30
topk_labs = asd_labs_freqs[:k].index

fig, ax = plt.subplots(figsize=(12,8), dpi=120)
sns.countplot(
    data=asd_labs.loc[asd_labs['STANDARD_CONCEPT_NAME'].isin(topk_labs)],
    y='STANDARD_CONCEPT_NAME',
    order = topk_labs,
    color='C0',
    ax=ax
)
ax.set(ylabel='Lab Concept');

In [ ]:
asd_labs.columns

In [ ]:
asd_labs[asd_labs['STANDARD_CONCEPT_NAME'].isin(topk_labs)].groupby('STANDARD_CONCEPT_NAME')['VALUE_AS_NUMBER'].describe()

# Modeling

## Loading

### Drug Exposures

In [ ]:
useful_cols = ['PERSON_ID', 'DRUG_CONCEPT_ID', 'STANDARD_CONCEPT_NAME', 'DRUG_EXPOSURE_START_DATETIME',
               'DRUG_EXPOSURE_END_DATETIME']

asd_drug = get_asd_drug()
asd_drug = asd_drug[useful_cols]

ctrl_drug = get_ctrl_drug()
ctrl_drug = ctrl_drug[useful_cols]

drug = pd.concat([asd_drug, ctrl_drug])

def add_drug_ancestors(df_drug, verbose=False):
    df_drug_anc = get_ancestors(df_drug['DRUG_CONCEPT_ID'])
    
    # Get nearest ancestor that is class C (a classification concept)
    df_drug_to_class = df_drug_anc.loc[(
        df_drug_anc[df_drug_anc['ancestor_class'] == 'C']
        .groupby('descendant_concept_id')['min_levels_of_separation']
        .idxmin()
    )]
    
    if verbose:
        # Summarize mapping
        n_start = df_drug['DRUG_CONCEPT_ID'].nunique()
        n_mapped = df_drug_to_class['descendant_concept_id'].nunique()
        n_end = df_drug_to_class['ancestor_concept_id'].nunique()
        print(f'Successfully mapped {n_mapped}/{n_start} ({n_mapped/n_start*100:.1f}%) of drug concepts to {n_end} higher order concepts.')

        n_exp = df_drug.shape[0]
        n_exp_mapped = df_drug['DRUG_CONCEPT_ID'].isin(df_drug_to_class['descendant_concept_id']).sum()
        n_exp_unmapped = n_exp - n_exp_mapped
        print(f'Unmapped drug concepts account for {n_exp_unmapped}/{n_exp} ({n_exp_unmapped/n_exp*100:.4f}%) of drug exposures.')

        ax = sns.histplot(x = df_drug[~df_drug['DRUG_CONCEPT_ID'].isin(df_drug_to_class['descendant_concept_id'])].groupby('DRUG_CONCEPT_ID').size(), discrete=True)
        ax.set(xlabel='Number of exposures', title='How many unmapped drug exposures are there?')
    
    # Reduce to mapped concepts and add ancestor mapping
    return (
        df_drug[df_drug['DRUG_CONCEPT_ID'].isin(df_drug_to_class['descendant_concept_id'])]
        .merge(
            df_drug_to_class[['descendant_concept_id', 'ancestor_concept_id', 'ancestor_name']],
            how='left',
            left_on='DRUG_CONCEPT_ID',
            right_on='descendant_concept_id'
        )
        .drop(columns=['descendant_concept_id'])
        .rename(columns=lambda x: x.upper())
    )

drug = add_drug_ancestors(drug)

drug.sort_values('PERSON_ID', inplace=True)

drug_on_asd = drug[drug['DRUG_CONCEPT_ID'].isin(asd_drug['DRUG_CONCEPT_ID'].unique())]

### Demographics

In [ ]:
nominal_cols = ['GENDER', 'RACE', 'ETHNICITY', 'SEX_AT_BIRTH']
asd_demo = get_asd_demo()
ctrl_demo = get_ctrl_demo()

asd_demo['AGE'] = (pd.Timestamp.now('UTC') - asd_demo['DATE_OF_BIRTH'])/pd.Timedelta('365 days')
ctrl_demo['AGE'] = (pd.Timestamp.now('UTC') - ctrl_demo['DATE_OF_BIRTH'])/pd.Timedelta('365 days')

asd_demo['COHORT'] = 'ASD'
ctrl_demo['COHORT'] = 'non-ASD'

demo = pd.concat([asd_demo, ctrl_demo])

gender_other = ['Not man only, not woman only, prefer not to answer, or skipped', 'No matching concept']
race_other = ['None of these', 'PMI: Skip', 'Another single population', 'I prefer not to answer', 'None Indicated']
ethn_other = ['What Race Ethnicity: Race Ethnicity None Of These', 'PMI: Skip', 'PMI: Prefer Not To Answer']
sex_birth_other = ['Not male, not female, prefer not to answer, or skipped', 'No matching concept']
demo.replace({
    'GENDER': gender_other,
    'RACE': race_other,
    'ETHNICITY': ethn_other,
    'SEX_AT_BIRTH': sex_birth_other}, 'Other', inplace=True)

demo = demo[demo['PERSON_ID'].isin(drug['PERSON_ID'].unique())].reset_index(drop=True)

def set_nominal(df, nominal_cols):
    for col in nominal_cols:
        cat = pd.CategoricalDtype(df[col].value_counts().index)
        df[col] = df[col].astype(cat)

set_nominal(demo, nominal_cols)

demo.sort_values('PERSON_ID', inplace=True)

In [ ]:
ctrl_demo['PERSON_ID'].isin(drug['PERSON_ID'].unique()).sum()

## Preprocessing

In [ ]:
nominal_cols = ['GENDER', 'RACE']

In [ ]:
nom_ohe = OneHotEncoder().fit(demo[nominal_cols])

In [ ]:
nom_fts = nom_ohe.transform(demo[nominal_cols])

In [ ]:
anc_ohe = OneHotEncoder().fit(drug[['ANCESTOR_CONCEPT_ID']])

In [ ]:
drug_ft_names = pd.merge(
    pd.Series(anc_ohe.categories_[0], name='ANCESTOR_CONCEPT_ID'),
    drug[['ANCESTOR_CONCEPT_ID', 'ANCESTOR_NAME']].drop_duplicates(),
    how='left'
)['ANCESTOR_NAME'].to_list()

In [ ]:
# Drug features for each subject, sorted by PERSON_ID. Long runtime!
# drug_cnt = sparse.vstack(
#     drug
#     .groupby('PERSON_ID')
#     .apply(lambda x: sparse.csr_matrix(anc_ohe.transform(x[['ANCESTOR_CONCEPT_ID']]).sum(0)))
# )
# sparse.save_npz('subj_drug_fts.npz', drug_cnt)

In [ ]:
drug_cnt = sparse.load_npz('subj_drug_fts.npz')

In [ ]:
age_col_idx = nom_fts.shape[1]
X = sparse.hstack([nom_fts, demo[['AGE']].to_numpy(), drug_cnt], 'csr')

nom_names = np.concatenate(nom_ohe.categories_).tolist()
nom_names[2] = 'Gender_' + nom_names[2]
nom_names[-2] = 'Race_' + nom_names[-2]
feature_names = pd.Series(nom_names + ['AGE'] + drug_ft_names)

cohort_enc = LabelEncoder()
y = cohort_enc.fit_transform(demo['COHORT'])

## Unpenalized LogReg on Drug Proportions

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y
)

In [ ]:
age_scaler = MaxAbsScaler().fit(X_train[:, age_col_idx])
X_train[:, age_col_idx] = age_scaler.transform(X_train[:, age_col_idx])

In [ ]:
drug_scaler = MaxAbsScaler().fit(X_train[:, age_col_idx+1:])
X_train[:, age_col_idx+1:] = drug_scaler.transform(X_train[:, age_col_idx+1:])

In [ ]:
X_test[:, age_col_idx] = age_scaler.transform(X_test[:, age_col_idx])
X_test[:, age_col_idx+1:] = drug_scaler.transform(X_test[:, age_col_idx+1:])

In [ ]:
logreg = LogisticRegression(
    solver='saga',
    penalty='none',
    class_weight='balanced',
    max_iter=1e2,
    random_state=0
)

logreg_clf = logreg.fit(X_train, y_train)

In [ ]:
print(clf_metrics(logreg_clf, X_train, X_test, y_train, y_test).to_latex())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
plot_roc_curve(logreg_clf, X_train, y_train, ax=ax)
ax.set(title='Training Data', xlabel='False Positive Rate (Positive: non-ASD)', ylabel='True Positive Rate (Positive: non-ASD)')
plt.tight_layout()
plt.savefig(os.path.join(dir_fig, 'train_roc.png'), dpi=300);

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_roc_curve(logreg_clf, X_test, y_test, ax=ax)
ax.set(title='Testing Data', xlabel='False Positive Rate (Positive: non-ASD)', ylabel='True Positive Rate (Positive: non-ASD)');
plt.tight_layout()
plt.savefig(os.path.join(dir_fig, 'test_roc.png'), dpi=300);

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_precision_recall_curve(logreg_clf, X_train, y_train, ax=ax)
ax.set(title='Training Data', ylim=[0, 1.01], xlabel='Recall (Positive: non-ASD)', ylabel='Precision (Positive: non-ASD)');
plt.tight_layout()
plt.savefig(os.path.join(dir_fig, 'train_prc.png'), dpi=300);

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_precision_recall_curve(logreg_clf, X_test, y_test, ax=ax)
ax.set(title='Testing Data', ylim=[0, 1.01], xlabel='Recall (Positive: non-ASD)', ylabel='Precision (Positive: non-ASD)');
plt.tight_layout()
plt.savefig(os.path.join(dir_fig, 'test_prc.png'), dpi=300);

In [ ]:
ft_importance = pd.DataFrame({
    'Feature Name': feature_names,
    'log Odds Ratio': logreg_clf.coef_.flatten()
})
ft_importance = ft_importance.loc[(-ft_importance['log Odds Ratio'].abs()).argsort()]
ft_importance.replace({'Feature Name': 'HYDROCODONE BITARTRATE AND ACETAMINOPHEN - hydrocodone bitartrate and acetaminophen syrup'}, 'Hydrocodone bitartrate and acetaminophen', inplace=True)

demo_importance = drug_importance = ft_importance[
    ~ft_importance['Feature Name'].isin(drug_ft_names)
]

drug_importance = ft_importance[
    ~ft_importance['Feature Name'].isin(np.concatenate(demo_ohe.categories_))
    &  (ft_importance['Feature Name'] != 'AGE')
]

In [ ]:
k = 20
fig, ax = plt.subplots(figsize=(20,12), dpi=120)
sns.barplot(data=ft_importance.head(k), x='log Odds Ratio', y='Feature Name', color='C0')
ax.set(xlabel='$\log$ Odds Ratio', ylabel='Feature Name')
plt.tight_layout()
plt.savefig(os.path.join(dir_fig, 'top20_fts_logreg_01.png'), dpi=300);

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
sns.barplot(data=demo_importance, x='log Odds Ratio', y='Feature Name', color='C0')
ax.set(xlabel='$\log$ Odds Ratio', ylabel='Feature Name');
plt.tight_layout()
plt.savefig(os.path.join(dir_fig, 'demo_fts_logreg_01.png'), dpi=300);

In [ ]:
k = 20
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
sns.barplot(data=drug_importance.head(k), x='log Odds Ratio', y='Feature Name', color='C0')
ax.set(xlabel='$\log$ Odds Ratio', ylabel='Feature Name');

## Unpenalized LogReg on Drug Indicators

In [ ]:
# drug_indicator = drug_cnt.copy()
# drug_indicator[drug_indicator.nonzero()] /= drug_indicator[drug_indicator.nonzero()]

In [ ]:
# X_indicator = sparse.hstack([nom_fts, demo[['AGE']].to_numpy(), drug_indicator], 'csr')

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=0, stratify=y
# )

In [ ]:
# age_scaler = MaxAbsScaler().fit(X_train[:, age_col_idx])
# X_train[:, age_col_idx] = age_scaler.transform(X_train[:, age_col_idx])

In [ ]:
# drug_scaler = MaxAbsScaler().fit(X_train[:, age_col_idx+1:])
# X_train[:, age_col_idx+1:] = drug_scaler.transform(X_train[:, age_col_idx+1:])

In [ ]:
# X_test[:, age_col_idx] = age_scaler.transform(X_test[:, age_col_idx])
# X_test[:, age_col_idx+1:] = drug_scaler.transform(X_test[:, age_col_idx+1:])

In [ ]:
logreg = LogisticRegression(
    solver='saga',
    penalty='none',
    class_weight='balanced',
    max_iter=1e3,
    random_state=0
)

logreg_clf = logreg.fit(X_ind_train, y_train)

In [ ]:
clf_metrics(logreg_clf, X_ind_train, X_ind_test, y_train, y_test)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
plot_roc_curve(logreg_clf, X_train, y_train, ax=ax)
ax.set(title='Training Data');

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
plot_roc_curve(logreg_clf, X_test, y_test, ax=ax)
ax.set(title='Testing Data');

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
plot_precision_recall_curve(logreg_clf, X_train, y_train, ax=ax)
ax.set(title='Training Data', ylim=[0, 1.01]);

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
plot_precision_recall_curve(logreg_clf, X_test, y_test, ax=ax)
ax.set(title='Testing Data', ylim=[0, 1.01]);

In [ ]:
ft_importance = pd.DataFrame({
    'Feature Name': feature_names,
    'log Odds Ratio': logreg_clf.coef_.flatten()
})
ft_importance = ft_importance.loc[(-ft_importance['log Odds Ratio'].abs()).argsort()]

demo_importance = drug_importance = ft_importance[
    ~ft_importance['Feature Name'].isin(drug_ft_names)
]

drug_importance = ft_importance[
    ~ft_importance['Feature Name'].isin(np.concatenate(demo_ohe.categories_))
    &  (ft_importance['Feature Name'] != 'AGE')
]

In [ ]:
feature_names.duplicated(keep=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
sns.barplot(data=demo_importance, x='log Odds Ratio', y='Feature Name', color='C0')
ax.set(xlabel='$\log$ Odds Ratio', ylabel='Feature Name');

In [ ]:
k = 30
fig, ax = plt.subplots(figsize=(12,12), dpi=120)
sns.barplot(data=drug_importance.head(k), x='log Odds Ratio', y='Feature Name', color='C0')
ax.set(xlabel='$\log$ Odds Ratio', ylabel='Feature Name');

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
plot_precision_recall_curve(logreg_clf, X_train, y_train, ax=ax)
ax.set(title='Training Data', ylim=[0, 1.01]);

In [ ]:
fig, ax = plt.subplots(figsize=(12,8), dpi=120)
plot_precision_recall_curve(logreg_clf, X_test, y_test, ax=ax)
ax.set(title='Testing Data', ylim=[0, 1.01]);

## Penalized Logistic Regression

In [ ]:
# For penalty, need to standardize since otherwise features with higher frequency
# will dominate penalty. -> standardization leads to dense matrix
# drug_scaler = StandardScaler().fit(drug_cnt)
# drug_fts = drug_scaler.transform(drug_cnt)

In [ ]:
# Only applicable if we want to use penalties
# search_res = RandomizedSearchCV(
#     estimator=logreg,
#     param_distributions=param_dists,
#     scoring='average_precision',
#     n_jobs=-1,
#     cv=5,
#     random_state=0
# )